Not all antibiotics that are produced by organisms follow the central dogma of biology (From **DNA** to **RNA** to **proteins**). Some antibiotics (*tyrocidines* and *gramicidins*) are non-ribosomal peptides, and they are not synthesized by ribosomes but by a protein called **NRP synthetase**. This enzyme can make antibiotic peptides without relying on **RNA**. Although these antibiotic peptides can have pharmaceutical applications, they cannot be sequenced from the DNA, but instead we have to rely on mass spectrometry to identify them.

With a mass spectrometer, the peptide can be broken into smaller pieces of which the mass can be generated. This leads to a mass spectrum and from this we would like to generate the sequence of the antibiotic peptide. This complex problem is slightly simplified in this notebook by assuming that amino acids have an integer mass and that the mass spectrum also only consists of integer masses.

Because the amino acids *isoleucine* and *leucine* both have an integer mass of 113 Da, and *lysine* and *glutamine* both have an integer mass of 128 Da, the duplicates will be removed.

In [9]:
# load in the integer mass table of amino acids
with open('integer_mass_table.txt') as f:
    mass_table = {}
    for line in f:
        line = line.strip('\n')
        new_line = line.split(' ')
        mass_table.update({new_line[0]: new_line[1]})

# remove the 2 amino acids because they have similar masses
del mass_table['I']
del mass_table['K']

The approach of this algorithm is that we keep increasing peptides and scoring these peptides with the amount of similarity to the provided spectrum. This allows for the handling of spectra with missing and/or false values which are often found in noisy real-life data. We keep track of the maximum score of the peptides and remove peptides that have a lower score. We also have to keep into account that these peptides can be circular!

To start this algorithm, we need a way to break a random peptide into a linear- or cyclic spectrum, and score this on the amount of mass matches with the provided spectrum.

In [10]:
# return the mass spectrum of all peptide fragments
def linear_spectrum(peptide, mass_table):
    prefix_mass = [0]
    alphabet = list(mass_table.keys())
    
    # the cumulative masses of the peptides
    for i in range(len(peptide)):
        for symbol in alphabet:
            if symbol == peptide[i]:
                prefix_mass.append(prefix_mass[i] + int(mass_table[symbol]))
    
    # calculate the mass differences of all the peptide fragments
    linear_spectrum = [0]
    for i in range(len(peptide)):
        for j in range(i+1, len(peptide)+1):
            linear_spectrum.append(prefix_mass[j]-prefix_mass[i])
    
    # sort the spectrum before returning
    return sorted(linear_spectrum)

# return the mass spectrum of all peptide fragments for a cyclical peptide
def cyclic_spectrum(peptide, mass_table):
    prefix_mass = [0]
    alphabet = list(mass_table.keys())
    
    # the cumulative masses of the peptides
    for i in range(len(peptide)):
        for symbol in alphabet:
            if symbol == peptide[i]:
                prefix_mass.append(prefix_mass[i] + int(mass_table[symbol]))
    
    # get the total peptide mass
    peptide_mass = 0
    for aa in peptide:
        peptide_mass += int(mass_table[aa])
    
    # calculate the mass differences of all the peptide fragments
    cyclic_spectrum = [0]
    for i in range(len(peptide)):
        for j in range(i+1, len(peptide)+1):
            cyclic_spectrum.append(prefix_mass[j]-prefix_mass[i])
            if i > 0 and j < len(peptide):
                cyclic_spectrum.append(peptide_mass - (prefix_mass[j]-prefix_mass[i]))
    
    # sort the spectrum before returning
    return sorted(cyclic_spectrum)

# this function scores a peptide with a given spectrum
def cyclopeptide_scoring(peptide, spectrum, cyclic, mass_table):
    # generate the spectrum of the peptide
    if cyclic == True:
        spectrum_to_compare = cyclic_spectrum(peptide, mass_table)
    elif cyclic == False:
        spectrum_to_compare = linear_spectrum(peptide, mass_table)

    # calculate the score (amount of matches with spectrum)
    score = 0
    spectrum_copy = spectrum.copy()
    for weight in spectrum_to_compare:
        if weight in spectrum_copy:
            score += 1
            spectrum_copy.remove(weight)

    return score

With these functions, we can break a random peptide into a spectrum and score this peptide (the higher the score, the better this peptide matches the spectrum). Next we want to see if the spectrum of the random peptide is consistent with the target spectrum. We also need to calculate the mass of the random peptide with the "peptide_mass" function.

In [11]:
# check if the peptide spectrum is consistent with the target spectrum
def consistent_spectrum(peptide, spectrum, mass_table):
    # generate the linear spectrum
    lin_spectrum = linear_spectrum(peptide, mass_table)

    # check if the masses are consistent
    for mass in lin_spectrum:
        if lin_spectrum.count(mass) > spectrum.count(mass):
            return False
    return True

# function to get the mass of a peptide
def peptide_mass(peptide, mass_table):
    mass = 0
    
    for ele in peptide:
        mass += int(mass_table[ele])
    
    return mass

For each round of the algorithm, we increase the size of random peptides and only keep the n-highest scoring peptides for further rounds. In this way, the algorithm does not have to continue with peptides that are not very well consistent with the spectrum. In order to be fair, we will also keep peptides that tie with the n-highest scoring peptides. For this we have the "trim" function.

In [12]:
# function to trim the leader_board to keep only the tied n peptides
def trim(leader_board, spectrum, n, mass_table):
    if len(leader_board) < n:
        final_leader_board = leader_board
    else:
        # make a leader_board dictionary
        linear_scores = dict.fromkeys(leader_board)
        # fill with linear scores
        for key in linear_scores.keys():
            score = cyclopeptide_scoring(key, spectrum, False, mass_table)
            linear_scores.update({key: score})
    
        # find the minimum score to be in the final leader_board
        scores = list(linear_scores.values())
        scores.sort(reverse = True)
        minimum_score = scores[n-1]
    
        # check for each peptide if they reach this minimum score
        final_leader_board = []
        for key, value in linear_scores.items():
            if value >= minimum_score:
                final_leader_board.append(key)
    
    return final_leader_board

Now we have all the components to generate a peptide from a measured spectrum. In each round, peptides are increased in size by adding all amino acids to the growing list of peptides. It is checked if the complete peptide mass is the highest mass in the spectrum, and if so, the peptides are scored and only the highest scoring peptide is kept.

In [13]:
# cyclopeptide sequencing using leader_board scoring
def leaderboard_peptide_sequencing(spectrum, n, mass_table):
    # the list of amino acids
    amino_acids = list(mass_table.keys())
    # the leader_board elements
    leader_board = [""]
    leader_peptide = ""
    leader_peptide_score = 0

    while leader_board:
        # expand the leader_board with new amino acids
        leader_board = [ele + aa for aa in amino_acids for ele in leader_board]
        peptides_to_remove = []
        for peptide in leader_board:
            mass = peptide_mass(peptide, mass_table)
            # if the mass is equal to the maximum mass check the score
            if mass == max(spectrum):
                # score the peptides
                score = cyclopeptide_scoring(peptide, spectrum, True, mass_table)
                # if this is the highest score make this the new leader peptide
                if score > leader_peptide_score:
                    leader_peptide = peptide
                    leader_peptide_score = score
            # else remove them from the leader_board
            elif mass > max(spectrum):
                # remove the peptide from candidate peptides
                peptides_to_remove.append(peptide)
        # remove all wrong candidates from the list
        for i in range(len(peptides_to_remove)):
            leader_board.remove(peptides_to_remove[i])
        # trim the leader_board before the next round unless it is already empty
        if len(leader_board) > 0:
            leader_board = trim(leader_board, spectrum, n, mass_table)
        else:
            break

    return leader_peptide

In this example we will use the spectrum of ***tyrocidine B1*** which has approximately 10% missing/false masses. As can be seen in the picture below, where blue masses are masses that are missing from the spectrum and green masses are false masses.

![example](tyrocidine_spectrum_10.png)

We can now apply our algorithm on this mass spectrum.

In [15]:
# read in the mass spectrum
with open("Mass_spectrum.txt", "r") as f:
    spectrum = f.read()
    spectrum = [int(i) for i in spectrum.split()]

# use the leaderboard sequencing algorithm
leader_peptide = leaderboard_peptide_sequencing(spectrum, 1000, mass_table)
print(f"This is the leader peptide: {leader_peptide}")

This is the leader peptide: WFNQYVQLFP


Now we have to take into account that the amino acids I/L and K/Q cannot be determined because they have similar integer masses.

The final predicted antibiotic peptide sequence of ***tyrocidine B1*** is then:
W F N K/Q Y V K/Q I/L F P

The amino acid sequence of this cyclic antibiotic is printed below.
![tyrocidine](tyrocidine_b1.png)

Or in one-letter codes:

(cyclo)- W F N Q Y V K L F P -(cyclo)

The peptide is thus correctly sequenced from the messy spectrum!

This notebook is a somewhat simplified version of messy real-life data. The current algorithm fails for example when the total peptide mass is not present in the spectrum. It also assumes that amino acids and the masses are integers, but this is something that can be quite easily implemented to account for (small) variations in masses in the spectrum. Furthermore, this spectrum is also not guaranteed to find the correct peptide sequence when too many masses are missing. 

Nevertheless, making the algorithm taught me a lot more about how to apply a scoring algorithm instead of brute-forcing a solution.